### FIle upload process for Russia's Balances for IEA.xlsx

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pysftp

sys.path.append('..')
sys.path.append('../iea_scraper')

In [3]:
import settings


In [6]:
from iea_scraper.settings import host, username, password
# from iea

In [7]:
def upload_file(from_path, to_path):
    """
    Uploads the file to the server.
    :param from_path: from local path.
    :param to_path: to path in the server.
    """
    cnopts = pysftp.CnOpts()
    cnopts.hostkeys = None
    print(f'Uploading file from {str(from_path)} to {to_path}')
    with pysftp.Connection(host=host, username=username, password=password, cnopts=cnopts) as srv:
        srv.put(from_path, to_path)
    os.remove(from_path)

In [8]:
import os
from pathlib import Path

current_dir = os.getcwd()
local_dir = Path(current_dir).parent / 'filestore' / 'ru_petromarket'

In [9]:
default_filename = "Russia's Balances for IEA.xlsx"
local_path = local_dir / default_filename
remote_path = f'/home/omrworker/projects/iea_scraper/filestore/ru_petromarket/{default_filename}'

In [10]:
remote_path

"/home/omrworker/projects/iea_scraper/filestore/ru_petromarket/Russia's Balances for IEA.xlsx"

In [11]:
# let's upload the file
# print(local_path, remote_path)
upload_file(local_path, remote_path)

Uploading file from C:\Users\DEGUCHI_T\PycharmProjects\scraper\filestore\ru_petromarket\Russia's Balances for IEA.xlsx to /home/omrworker/projects/iea_scraper/filestore/ru_petromarket/Russia's Balances for IEA.xlsx


### Scratch Paper

In [1]:
import pandas as pd
import openpyxl

In [8]:
file = 'C:/Users/DEGUCHI_T/PycharmProjects/scraper/filestore/ru_petromarket/Russia\'s Balances for IEA.xlsx'

In [9]:
df_sheet_multi = pd.read_excel(file, sheet_name=['OIL', 'Condensate', 'LPG', 'GASOLINE', 'NAPHTHA', 'JET FUEL', 'GASOIL', 'VGO', 'FUEL OIL', 'COKE'], index_col=0)

In [10]:
df_oil = df_sheet_multi['OIL'].iloc[[1,2,5,6,10,11,12,13],:].reset_index().copy()
df_oil.rename(columns={'RUSSIA\'s CRUDE OIL(*) BALANCE (thou. tonnes)':'category'}, inplace=True)

df_oil['demand_supply'] = ['Supply', 'Supply', 'Demand', 'Demand', 'Demand', 'Demand', 'Demand', 'Demand']
df_oil['category'] = ['Production', 'Imports', 'InsideDemand', 'InsideDemand', 'Exports', 'Exports', 'Exports', 'Exports']
df_oil['subcategory'] = ['', '', 'Throughput', 'Use/Loss', 'CrudeOil', 'CrudeOil', 'CrudeOil', 'Condensate']
df_oil['area'] = ['', '', '', '', 'CISWest', 'CISEast', 'CISInside', '']

df_oil.set_index(['demand_supply', 'category', 'subcategory', 'area'], inplace=True)
df_oil_tmp = df_oil.stack().reset_index()
df_oil_tmp.rename(columns={'level_4': 'date', 0: 'value'}, inplace=True)

In [11]:
df_condensate = df_sheet_multi['Condensate'].iloc[[2,3,5,6,7],:].reset_index().copy()
df_condensate.rename(columns={'RUSSIAN GAS CONDENSATE SUPPLY AND DEMAND BALANCE (thou. tonnes)':'category'}, inplace=True)

df_condensate['demand_supply'] = ['Supply', 'Supply', 'Demand', 'Demand', 'Demand']
df_condensate['category'] = ['Production', 'Imports', 'InsideDemand', 'InsideDemand', 'Exports']
df_condensate['subcategory'] = ['', '', 'Throughput', 'Use/Loss', '']
df_condensate['area'] = ['', '', '', '', '']

df_condensate.set_index(['demand_supply', 'category', 'subcategory', 'area'], inplace=True)
df_condensate_tmp = df_condensate.stack().reset_index()
df_condensate_tmp.rename(columns={'level_4': 'date', 0: 'value'}, inplace=True)

In [12]:
df_diesel = df_sheet_multi['GASOIL'].iloc[[1,2,4,5],:].reset_index().copy()
df_diesel.rename(columns={'RUSSIA\'s DIESEL BALANCE (thou. tonnes)':'category'}, inplace=True)

df_diesel['demand_supply'] = ['Supply', 'Supply', 'Demand', 'Demand']
df_diesel['category'] = ['Production', 'Imports', 'Consumption', 'Exports']
df_diesel['subcategory'] = ['', '', '', '']
df_diesel['area'] = ['', '', '', '']

df_diesel.set_index(['demand_supply', 'category', 'subcategory', 'area'], inplace=True)
df_diesel_tmp = df_diesel.stack().reset_index()
df_diesel_tmp.rename(columns={'level_4': 'date', 0: 'value'}, inplace=True)

In [13]:
df_heating_oil = df_sheet_multi['GASOIL'].iloc[[15,16,18,19],:].reset_index().copy()
df_heating_oil.rename(columns={'RUSSIA\'s DIESEL BALANCE (thou. tonnes)':'category'}, inplace=True)

df_heating_oil['demand_supply'] = ['Supply', 'Supply', 'Demand', 'Demand']
df_heating_oil['category'] = ['Production', 'Imports', 'Consumption', 'Exports']
df_heating_oil['subcategory'] = ['', '', '', '']
df_heating_oil['area'] = ['', '', '', '']

df_heating_oil.set_index(['demand_supply', 'category', 'subcategory', 'area'], inplace=True)
df_heating_oil_tmp = df_heating_oil.stack().reset_index()
df_heating_oil_tmp.rename(columns={'level_4': 'date', 0: 'value'}, inplace=True)

In [78]:
def transform_other_poduct(product :str,df_tmp: pd.DataFrame) -> pd.DataFrame:

    df_tmp = df_sheet_multi[product].iloc[[1,2,4,5],:].reset_index().copy()
    
    if product == 'GASOLINE':
        title = 'MOTOR GASOLINE'
    elif product == 'COKE':
        title = 'COKES'
    else:
        title = product
    df_tmp.rename(columns={f'RUSSIA\'s {title} BALANCE (thou. tonnes)':'category'}, inplace=True)    

    
    df_tmp['demand_supply'] = ['Supply', 'Supply', 'Demand', 'Demand']
    df_tmp['category'] = ['Production', 'Imports', 'Consumption', 'Exports']
    df_tmp['subcategory'] = ['', '', '', '']
    df_tmp['area'] = ['', '', '', '']
   
    df_tmp.set_index(['demand_supply', 'category', 'subcategory', 'area'], inplace=True)
    df_tmp_transpose = df_tmp.stack().reset_index()
    df_tmp_transpose.rename(columns={'level_4': 'date', 0: 'value'}, inplace=True)

    df_tmp_transpose['product'] = product 
    
    df_tmp_transpose = df_tmp_transpose.reindex(columns=['product', 'demand_supply', 'category', 'subcategory', 'area', 'date', 'value'])
    
    return df_tmp_transpose

In [141]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [142]:
def transpose_other(product : str, df_tmp : pd.DataFrame):
    """
    Transpose df to tabular form for other products.

    :param df_tmp:
    :return:
    """

    df_other = df_tmp.iloc[[1, 2, 4, 5], :].reset_index().copy()

    if product == 'GASOLINE':
        title = 'MOTOR GASOLINE'
    elif product == 'COKE':
        title = 'COKES'
    else:
        title = product
    df_other.rename(columns={f'RUSSIA\'s {title} BALANCE (thou. tonnes)': 'category'}, inplace=True)

    df_other['demand_supply'] = ['Supply', 'Supply', 'Demand', 'Demand']
    df_other['category'] = ['Production', 'Imports', 'Consumption', 'Exports']
    df_other['subcategory'] = ['', '', '', '']
    df_other['area'] = ['', '', '', '']
    
    df_other.set_index(['demand_supply', 'category', 'subcategory', 'area'], inplace=True)    

    df_other_return = df_other.stack().reset_index()

    df_other_return.rename(columns={'level_4': 'date', 0: 'value'}, inplace=True)
    
    df_other_return['product'] = product

    df_other_return = df_other_return.reindex(
        columns=['product', 'demand_supply', 'category', 'subcategory', 'area', 'date', 'value'])

#     logger.info(f'Return {product} counts : {len(df_other_return)} rows.')

    return df_other_return


In [143]:
test = transpose_other('COKE',df_sheet_multi['COKE'])

In [144]:
test.to_csv('out.csv')

In [ ]:

# let's rename it
new_name = f'cn_customs_crudeoil_imp_{selected_year}{int(start_month):02d}.csv'
new_path = current_path.with_name(new_name)
current_path.rename(new_path)

# let's upload the file
remote_path = f'/home/omrworker/projects/iea_scraper/filestore/{new_name}'
utils.upload_file(new_path, remote_path)

print('Successfully finished.')




In [ ]:
def upload_file(from_path, to_path):
    """
    Uploads the file to the server.
    :param from_path: from local path.
    :param to_path: to path in the server.
    """
    cnopts = pysftp.CnOpts()
    cnopts.hostkeys = None
    print(f'Uploading file from {str(from_path)} to {to_path}')
    with pysftp.Connection(host=host, username=username, password=password, cnopts=cnopts) as srv:
        srv.put(from_path, to_path)
    os.remove(from_path)

